### 一. volatile
#### 1. 线程共享变量模型   
1. Java中, 每个线程都有自己独立的执行内存空间.   
2. 当线程需要存取一个共享变量时, 要先获取锁定, 并从主内存拷贝共享变量到自己的线程执行空间里  
当线程操作完这个变量后, 释放锁并将结果写会主内存  

#### 2. 线程安全问题  
假设一个线程从共享内存中拷贝了共享变量后释放了锁, 而后线程内部操作该变量. 此时, 有另一个线程又从主内存拷贝共享变量, 这个新线程拷贝的结果会是上一个线程没做出修改之前的值, 从而产生安全问题  

#### 3. `volatile`关键字  
volatile关键字声明的变量可以做到如下保证 :  
1. 其他线程对共享变量的修改, 可以即时反应在当前线程中  
2. 确保当前线程对volatile变量的修改, 能即时写回到 共享主内存中, 并被其他线程看见  
3. volatile声明的变量, 将破事所有线程均读写主内存中的变量值, 从而使得volatile变量在多线程间可见  

#### 4. -server模式  
java在'-server'模式下, JVM不回去读取未发生变化且未标记为volatile的共享变量, 使得变量的修改在线程之间是不可见的

#### 5. volatile与指令重拍  
&nbsp;&nbsp;&nbsp;&nbsp;编译器以及CPU为了优化代码执行的效率, 可能会使得本来在代码上发生的先执行a再执行b, 优化后变成先执行b再执行a; 这在单线程程序中不会发生异常, 但是在多线程环境中可能破坏了操作之间的happens-before限制, 导致发生位置问题.  
&nbsp;&nbsp;&nbsp;&nbsp;使用volatile关键字后, 除了提供内存可见性语义, 还会避免volatile附近的代码发生指令重排序, 使得线程安全; volatile的缺点是变量只会存在内存中, 而不会进入寄存器等不可见地方, 一定程度上降低读写效率

### 二. Synchronized
#### 1. Synchronized使用方法  
 1. **类级别的锁**  (所有同步针对该类的所有对象)
    1. 同步静态方法   
    同步静态方法是类级别的锁，一旦任何一个线程进入这个方法，其他所有线程将无法访问这个类的任何同步类锁的方法。
        ```java
        public synchronized static void fun() { }
        ```
    2. 同步代码块锁类  
    下面提供了两种同步类的方法，锁住效果和同步静态方法一样，都是类级别的锁，同时只有一个线程能访问带有同步类锁的方法。
        ```java
        private void fun() {
            synchronized (this.getClass()) { }
        }
        ```
    
 2. **对象计别的锁** (所有同步只针对同一个对象)
    1. 同步普通方法
        ```java
        public synchronized void fun() { }
        ```
    2. 同步代码块中使用this对象/其它对象作为锁
        ```java
        public void fun() {
            synchronized (this) { }
        }

        public void fun() {
            synchronized (LOCK) { }
        }
        ```
        
#### 2. synchronized与wait,notify合用  
 单一的synchronized虽然可以保证线程安全, 但需要配合其它线程方法, 才能表示复杂逻辑的线程交互  
 1. `obj.wait()`  
    1. 使用方法  
     ```java
         synchronized(obj){
            while(条件){
                obj.wait();
                // 收到通知后, 继续执行
            }
        }
     ```
    2. 使用wait()之前, 需要获取对象锁.   
    3. 其次, wait()方法要写在while循环中, 并指明跳出循环的条件 :  
        因为wait()别唤醒后, 原先的判断条件可能已经发生改变, 需要再次判断  
    4. 最后, wait()方法执行时, 线程会释放得到的obj独占锁, 并进入'等待阻塞'状态, 等待其它线程执行该`obj`锁的`notify()` 
 2. `obj.notify()`  
   当等待在obj上的线程收到一个`obj.notify()`时, 就能重新获得obj的锁.值得注意的是以下3点 :  
    1. 当线程执行完`obj.notify()`后, 不会立刻释放锁, 而是等待synchronized代码块中的代码全部执行完毕后再释放锁  
    2. 如果有多个线程在方法`obj.wait()`中, 则只会随机选择一个线程唤醒  
    3. `obj.notifyAll()`会唤醒所有在执行`obj.wait()`的线程
 3. `Thread.sleep`    
  sleep方法会让线程休眠, 但不会释放已获得的锁 

### 三. 显式锁
